### Decorating Generator Functions

```
def gen(args):
    # set up happens here, or inside try
    try:
        yield obj  # whatever normally gets returned by __enter__
    finally:
        # perform clean up code here
        

In [20]:
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()
        print('file closed.')

In [21]:
class GenContextManager:
    def __init__(self, gen):
        self.gen = gen
        
    def __enter__(self):
        print('calling next to get the yielded value from generator')
        return next(self.gen)
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('calling next to perform cleanup in generator')
        try:
            next(self.gen)
        except StopIteration:
            pass
        return False
    

In [22]:
file_gen = open_file('test.txt', 'w')
with GenContextManager(file_gen) as f:
    f.writelines('Sir Spamalot')

calling next to get the yielded value from generator
opening file...
calling next to perform cleanup in generator
closing file...
file closed.


#### Context Manager Decorator

In [23]:
def context_manager_dec(gen_fn):
    def helper(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        
        # Instantiate
        ctx = GenContextManager(gen)
        return ctx
    
    return helper


In [24]:
@context_manager_dec
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()
        print('file closed.')
        
        
# open_file = context_manager_dec(open_file)


In [25]:
with open_file('test.txt') as f:
    print(f.readlines())

calling next to get the yielded value from generator
opening file...
['Sir Spamalot']
calling next to perform cleanup in generator
closing file...
file closed.


#### contextlib module

In [26]:
from contextlib import contextmanager

In [27]:
@contextmanager
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()
        print('file closed.')
        

In [28]:
with open_file('test.txt', 'r') as f:
    print(f.readlines())

opening file...
['Sir Spamalot']
closing file...
file closed.


In [29]:
from time import perf_counter, sleep


In [30]:
@contextmanager
def timer():
    stats = dict()
    start = perf_counter()
    
    stats['start'] = start
    
    try:
        yield stats
    finally:
        end = perf_counter()
        stats['end'] = end
        stats['elapsed'] = end - start
        

In [31]:
with timer() as stats:
    sleep(3)

In [32]:
print(stats)

{'start': 80330.4660849, 'end': 80333.4663534, 'elapsed': 3.0002684999926714}


In [33]:
import sys

In [34]:
@contextmanager
def out_to_file(fname):
    curr_stdout = sys.stdout
    file = open(fname, 'w')
    sys.stdout = file
    
    try:
        yield None
    finally:
        file.close()
        sys.stdout = curr_stdout
        

In [35]:
with out_to_file('test.txt'):
    print('line 1')
    print('line 2')
    print('line 3')

In [36]:
print('test')

test


In [37]:
with open('test.txt') as f:
    print(f.readlines())

['line 1\n', 'line 2\n', 'line 3\n']


In [38]:
from contextlib import redirect_stdout

In [39]:
with open('test.txt', 'w') as f:
    with redirect_stdout(f):
        print('Look on the bright side of life')
        

In [40]:
with open('test.txt', 'r') as f:
    print(f.readlines())

['Look on the bright side of life\n']
